**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°:1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Lectura de DF

Descargamos los dataset desde el link, para no lidiar con inconvenientes al abrir y cerrar sesiones de colab.

In [3]:
!wget -O bgg_database.csv 'https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq'
!wget -O IMDB-Movie-Data.csv 'https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c'

--2024-10-30 12:51:06--  https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.google.com (drive.google.com)... 172.217.7.46, 2607:f8b0:4025:811::200e
Connecting to drive.google.com (drive.google.com)|172.217.7.46|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq [following]
--2024-10-30 12:51:07--  https://drive.usercontent.google.com/download?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.0.65, 2607:f8b0:4025:810::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.0.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1833113 (1.7M) [application/octet-stream]
Saving to: ‘bgg_database.csv’

bgg_database.csv    100%[===================>]   1.75M  --.-KB/s    in 0.1s    

2024-10-30 12:51:09 (15.9 MB/s) - ‘bgg_databas

In [4]:
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')
df_peliculas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')

### Exploración y procesado de df_juegos

Tomar criterios de evaluación para ver si borrar columnas o no.

In [43]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxplayers        1000 non-null   int64  
 10  minplaytime       1000 non-null   int64  
 11  maxplaytime       1000 non-null   int64  
 12  minage            1000 non-null   int64  
 13  avgweight         1000 non-null   float64
 14  best_num_players  1000 non-null   object 
 15  designers         1000 non-null   object 
 16  mechanics         1000 non-null   object 
 

Podemos visualizar que las siguientes columnas presentan datos que neceistamos trabajar:
- best_num_players: presenta una lista que contiene un diccionario.
- designers, mechanics, categories: son listas.
- description: tiene caracteres '\n'

In [44]:
df_juegos.sample(5)

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
862,863,A War of Whispers,https://boardgamegeek.com/boardgame/253499/a-w...,6.700,7.32,4327.0,A War of Whispers is a competitive board game ...,2019,2,4,30,60,14,2.5766,"[{'min': 4, 'max': 4}]",['Jeremy Stoltzfus'],"['Area Majority / Influence', 'Betting and Blu...","['Fantasy', 'Political', 'Spies/Secret Agents'..."
684,685,Gears of War: The Board Game,https://boardgamegeek.com/boardgame/42776/gear...,6.820,7.47,5114.0,Game description from the publisher:\n\nFor a ...,2011,1,4,60,180,13,2.8750,"[{'min': 2, 'max': 2}, {'min': 4, 'max': 4}]",['Corey Konieczka'],"['Area Movement', 'Campaign / Battle Card Driv...","['Fighting', 'Miniatures', 'Science Fiction', ..."
449,450,Chess,https://boardgamegeek.com/boardgame/171/chess,7.014,7.21,36841.0,"Chess is a two-player, abstract strategy board...",1475,2,2,0,0,6,3.6536,"[{'min': 2, 'max': 2}]",['(Uncredited)'],"['Grid Movement', 'Once-Per-Game Abilities', '...",['Abstract Strategy']
738,739,Brian Boru: High King of Ireland,https://boardgamegeek.com/boardgame/337765/bri...,6.775,7.51,3696.0,"In Brian Boru: High King of Ireland, you striv...",2021,3,5,60,90,14,2.5000,"[{'min': 4, 'max': 4}]",['Peer Sylvester'],"['Area Majority / Influence', 'Closed Drafting...","['Card Game', 'Medieval', 'Territory Building']"
367,368,Bonfire,https://boardgamegeek.com/boardgame/304420/bon...,7.100,7.65,6595.0,"The bonfires are sources of light, energy and ...",2020,1,4,70,100,12,3.6276,"[{'min': 3, 'max': 3}]",['Stefan Feld'],"['Market', 'Pattern Building', 'Point to Point...","['Fantasy', 'Puzzle']"


In [45]:
df_juegos.describe()

,rank,geek_rating,avg_rating,num_voters,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,7.053187,7.570630,13615.758000,2008.427000,1.760000,4.937000,59.55700,93.397000,11.636000,2.680625
std,288.819436,0.351866,0.391122,15876.211424,135.395031,0.689112,6.272509,43.50627,83.159816,2.252683,0.819375
min,1.000000,6.615000,6.760000,1364.000000,-2200.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000
25%,250.750000,6.770000,7.277500,4608.250000,2011.000000,1.000000,4.000000,30.00000,45.000000,10.000000,2.100300
50%,500.500000,6.970000,7.530000,7983.500000,2016.000000,2.000000,4.000000,60.00000,77.500000,12.000000,2.642300
75%,750.250000,7.256250,7.802500,16097.750000,2019.000000,2.000000,5.000000,60.00000,120.000000,14.000000,3.266025
max,1000.000000,8.415000,9.040000,128735.000000,2024.000000,6.000000,100.000000,480.00000,1200.000000,18.000000,4.818200


In [48]:
df_juegos.isna().sum() #no encontramos nulos.

,0
rank,0
game_name,0
game_href,0
geek_rating,0
avg_rating,0
num_voters,0
description,0
yearpublished,0
minplayers,0
maxplayers,0


### Exploración y procesado de df_peliculas

In [46]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  1000 non-null   float64
 11  Metascore           1000 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [47]:
df_peliculas.sample(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
548,549,Chuck,"Biography,Drama,Sport",A drama inspired by the life of heavyweight bo...,Philippe Falardeau,"Elisabeth Moss, Naomi Watts, Ron Perlman, Liev...",2016,98,6.8,391,0.11,68
946,947,The Butler,"Biography,Drama",As Cecil Gaines serves eight presidents during...,Lee Daniels,"Forest Whitaker, Oprah Winfrey, John Cusack, J...",2013,132,7.2,93322,116.63,66
507,508,(500) Days of Summer,"Comedy,Drama,Romance",An offbeat romantic comedy about a woman who d...,Marc Webb,"Zooey Deschanel, Joseph Gordon-Levitt, Geoffre...",2009,95,7.7,398972,32.39,76
278,279,Everybody Wants Some!!,Comedy,"In 1980, a group of college baseball players n...",Richard Linklater,"Blake Jenner, Tyler Hoechlin, Ryan Guzman,Zoey...",2016,117,7.0,36312,3.37,83
965,966,Inland Empire,"Drama,Mystery,Thriller",As an actress starts to adopt the persona of h...,David Lynch,"Laura Dern, Jeremy Irons, Justin Theroux, Karo...",2006,180,7.0,44227,0.00,72


In [49]:
df_peliculas.isna().sum() # no encontramos nulos.

,0
Rank,0
Title,0
Genre,0
Description,0
Director,0
Actors,0
Year,0
Runtime (Minutes),0
Rating,0
Votes,0


### df_libros: web scrapping y procesado


In [5]:
url: str = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

In [6]:
response: requests.models = requests.get(url)

# extraigo el contenido de la página web
soup: BeautifulSoup = BeautifulSoup(response.text, 'html.parser')

# luego de la inspección, vemos que necesitamos la información de page_content
libro_div = soup.find('div', class_='page_content')

# divido para quedarme con los títulos
paragraphs = libro_div.find_all('a')

In [7]:
# creo lista para poder crear el df
libro: list = [par.text for par in paragraphs]
libro = libro[1::] # mediante slicing salteo la primera linea con información que no nos interesa.

In [8]:
df_libros: pd.DataFrame = pd.DataFrame(libro, columns=['Libro'])

In [9]:
def limpieza_top(libro: str) -> bool:
  """
  Chequea si en string existe el substring 'Top 1000'. Retorna un booleano
  """
  if 'Top 1000' in libro:
    return True
  else:
    return False

In [10]:
df_libros['es_ruido'] = df_libros['Libro'].apply(limpieza_top)

In [11]:
df_libros.drop(df_libros[df_libros['es_ruido'] == True].index, inplace=True)

In [12]:
df_libros = df_libros.drop('es_ruido', axis=1) # dropeo columna auxiliar.

In [13]:
df_libros.reset_index(drop=True, inplace=True) # reseteo indices, ya que eliminé las filas que no son libros.

In [14]:
df_libros = df_libros.drop_duplicates() # elimino duplicados si los hubiera

In [39]:
df_libros_filtered: pd.DataFrame = df_libros.copy()
df_libros_filtered['Libro'] = df_libros['Libro'].str.replace(r'\s*\(\d+\)$', '', regex=True)